# Image Recognition

Verrà utilizzato YOLO nella versione 11 con modello nano. 

## ✅ Step 1 – Preparazione
Assicurarsi che la libreria `ultralytics` sia installata

In [ ]:
%pip install --quiet ultralytics

``base_dir`` rappresenta il percorso assoluto del dataset mentre ``workers`` rappresenta il numero di thread che il progetto potrà usare quando incontra funzioni multithread. Attenzione che fa un sacco di overhead se si imposta il parametro ``workers`` di YOLO ad un valore elevato.

In [1]:
import os 
import json

base_dir = "/mnt/e/objects365"
workers = max(4, os.cpu_count() - 2) # 22 sul mio PC

Il dataset scaricato non ha la cartella di test ma solo `train` e `val` che sono per appunto addestramento e validazione. Ho deciso di creare una cartella di test togliendo dalla cartella di validazione alcune patch. 

In [ ]:
from data.create_test_samples import create_test_set

create_test_set(base_dir, test_size_ratio=0.1, num_threads=8)

Le patch sono molto grandi e per limiti hardware ho deciso di fare uno split, quindi sia labels che images che test avranno il doppio delle patch ma grandi la metà dove ogni patchX sarà divisa in patchX_a e patchX_b doce la X è un numero intero. Col senno di poi non è utilissimo per il nuovo approccio ma comunque aiuta perché un limite spesso è dato dal filesystem e dalla sua capacità di gestire grandi moli di dati per singola cartella e in tal senso NTFS non eccelle, quindi comunque lavoro che potrebbe aver avuto la sua utilità.

In [ ]:
from data.split_patch import dividi_patch_in_due_multithread
import os

workers = max(4, os.cpu_count() - 2)

# Gestisco le Labels
dividi_patch_in_due_multithread(f"{base_dir}/labels/test", max_workers=workers)
dividi_patch_in_due_multithread(f"{base_dir}/labels/val", max_workers=workers)
dividi_patch_in_due_multithread(f"{base_dir}/labels/train", max_workers=workers)

# Gestisco le immagini
dividi_patch_in_due_multithread(f"{base_dir}/images/test", max_workers=workers)
dividi_patch_in_due_multithread(f"{base_dir}/images/train", max_workers=workers)
dividi_patch_in_due_multithread(f"{base_dir}/images/val", max_workers=workers)


Siccome YOLO ha bisogno di un file di configurazione in formato YAML (Yet Another Markup Language) e siccome il dataset è diviso in patch c'è bisogno di dare le coordinate delle patch a YOLO e per farlo bisogna creare un file TXT con tutti i percorsi assoluti di tutti i file da processare. Nel frattempo vista la grande mole del dataset eseguo un subsampling abbastanza drastico per non incorrere il limiti hardware. 

In [27]:
from data.create_image_lists import create_image_lists_with_subsampling


# Crea gli elenchi delle immagini per il training, la validazione e il test
%load_ext autoreload
%autoreload 2
# --- Configuration ----
source_dataset_base_path = "/mnt/e/objects365"
destination = "/mnt/e/objects365"
splits = ["train", "val", "test"]
target_samples_per_class=300
filename = f"{splits[0]}_subsampling_{target_samples_per_class}.txt"


print(f"Creating image lists for split: {splits[0]}")
create_image_lists_with_subsampling(
    base_dir=source_dataset_base_path,
    split=splits[0],
    output_file=os.path.join(destination, filename),
    target_samples_per_class=target_samples_per_class,
    save_class_stats=True
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Creating image lists for split: train

🚀 Starting image list creation for 'train' split with subsampling...
📁 Base directory: /mnt/e/objects365
💾 Output file: /mnt/e/objects365/train_subsampling_300.txt

📋 Step 1/4: Discovering and reading labels...
🔍 Scanning labels in: /mnt/e/objects365/labels/train
📂 Found 102 patch directories
⚙️  Processing with 22 workers...


🔄 Processing patches:   0%|          | 0/102 [00:00<?, ?patch/s]

⚠️  Warning: Image not found for label file /mnt/e/objects365/labels/train/patch16_b/objects365_v2_00919985.txt at /mnt/e/objects365/images/train/patch16_b/objects365_v2_00919985.jpg
⚠️  Warning: Image not found for label file /mnt/e/objects365/labels/train/patch16_a/objects365_v2_00908726.txt at /mnt/e/objects365/images/train/patch16_a/objects365_v2_00908726.jpg


🔄 Processing patches:  88%|████████▊ | 90/102 [26:23<05:41, 28.50s/patch, TXT files=1535090, Images found=1535088, Classes=365]

⚠️  Warning: Image not found for label file /mnt/e/objects365/labels/train/patch6_b/objects365_v1_00320532.txt at /mnt/e/objects365/images/train/patch6_b/objects365_v1_00320532.jpg
⚠️  Warning: Image not found for label file /mnt/e/objects365/labels/train/patch6_b/objects365_v1_00320534.txt at /mnt/e/objects365/images/train/patch6_b/objects365_v1_00320534.jpg


...


✅ Finished collecting all labels!
📊 Summary:
   • Processed patches: 102/102
   • Total label files: 1742292
   • Total images found: 1742287
   • Total classes discovered: 365

📋 Step 2/4: Performing subsampling...

🎯 Starting subsampling...
📈 Dataset overview:
   • Total classes: 365
   • Total images: 25407576
   • Top 10 classes by count: {1: 6004123, 2: 1243038, 3: 1015094, 4: 923582, 5: 857935, 6: 739122, 7: 696076, 8: 674616, 9: 627705, 10: 539673}
🎯 Targeting 300 samples per class...


🔄 Subsampling classes: 100%|██████████| 365/365 [00:00<00:00, 1265.33class/s, Selected images=93507]



✅ Subsampling complete!
📊 Results:
   • Selected 93507 unique images
   • Reduction: 25407576 → 93507 (0.4%)

📋 Step 3/4: Saving image paths...
💾 Writing 93507 paths to file...


💾 Writing paths: 100%|██████████| 93507/93507 [00:00<00:00, 417204.88path/s]


✅ Saved 93507 image paths to: /mnt/e/objects365/train_subsampling_300.txt

🎉 Image list creation completed successfully!
📊 Final summary:
   • Split: train
   • Total images selected: 93507
   • Output file: /mnt/e/objects365/train_subsampling_300.txt


Qui un estratto del file YAML:

```yaml
path: /mnt/e/objects365 # Percorso base
train: train_subsampling_300.txt  # File con elenco immagini di training
val: val_subsampling_100.txt      # File con elenco immagini di validation
test: test_subsampling_100.txt    # File con elenco immagini di test
```


Mi sono accorto che nelle etichette spesso compare la classe 365 ma questa non è mappata nel file YAML e siccome YOLO non ha la classe `background` come default per le classi sconosciute allora per identificare tutto ciò che non è mappabile dal modello, ho deciso di mappare la classe 365 delle etichette come classe `Unknown`.

```yaml
# Classes
names:
  0: Person
  1: Sneakers
  2: Chair
  3: Other Shoes
  ...
  361: Lipstick
  362: Cosmetics Mirror
  363: Curling
  364: Table Tennis
  365: Unknown
```

## ✅ Step 2 – Addestramento del Modello

Il modello di YOLO scelto sarà un *pretrained*, successivamente verrà importato il modello addestrato sulle patch del dataset. Ho scelto il modello nano perché compatibile insieme alla dimensine delle patch per girare sulla mia GPU, anche solo con il modello small la quantità di VRAM necessaria eccede quella della mia GPU e andando ad utilizzare della memoria condivisa in RAM il processo rallenta drasticamente al punto di passare da iterazioni al secondo a secondi per iterazione.


In [2]:
from ultralytics import YOLO

# Load a model
model_name = "yolo11s.pt"  # or any other YOLO model
model = YOLO(model_name)  # load a pretrained model (recommended for training)

## Augmentation Parameters
Gli aumenti dei dati sono tecniche utilizzate per migliorare la robustezza e la generalizzazione del modello. Ecco i parametri configurabili:

- **hsv_h (Hue)**: Regola la tonalità dei colori per rendere il modello invariante alle variazioni di colore.
- **hsv_s (Saturation)**: Modifica la saturazione per simulare diverse condizioni di illuminazione.
- **hsv_v (Brightness)**: Cambia la luminosità per gestire immagini con diverse esposizioni.
- **degrees**: Applica rotazioni casuali per migliorare l'invarianza al punto di vista.
- **translate**: Introduce traslazioni casuali per rendere il modello robusto a oggetti in posizioni diverse.
- **scale**: Ridimensiona gli oggetti per gestire variazioni di dimensioni.
- **shear**: Applica tagli (shearing) per simulare distorsioni geometriche (impostato a 0 per evitare distorsioni estreme).
- **fliplr**: Effettua un flip orizzontale casuale per aumentare la varietà dei dati.
- **mosaic**: Combina più immagini in un'unica immagine per migliorare il rilevamento di piccoli oggetti e il contesto.
- **bgr**: Scambia i canali BGR per gestire diversi formati di immagine.

In [3]:
data_yaml = "data/objects365.yaml"
run = f"runs/train_{model_name.split('.')[0]}"
train_params = {
    "data":         data_yaml,               # YAML con path /mnt/e/yolo-fast
    "epochs":       10,                      # numero di epoche
    "patience":     2,                       # early stopping
    "batch":        32,                      # auto batch size (~60% VRAM)
    "imgsz":        640,                     # dimensione delle immagini
    "device":       0,                       # GPU 0
    "workers":      4,                       # DataLoader parallelismo
    "cache":        "disk",                  # caching su disco (/mnt/e)
    "save":         True,                    # salva checkpoint e modello
    "save_period":  5,                       # checkpoint ogni 5 epoche
    "project":      run,                     # cartella di uscita
    "name":         "exp_fastcache",         # nome del run
    "exist_ok":     True,                    # sovrascrivi se esiste
    "pretrained":   True,                    # usa pesi pretrained
    "optimizer":    "AdamW",                 # SGD o Adam/AdamW
    "amp":          True,                    # mixed precision
    "rect":         False,                   # rectangular training off
    "multi_scale":  False,                    # multi-scale training on
    "cos_lr":       True,                    # cosine LR scheduler
    "close_mosaic": 10,                      # disabilita mosaic ultime 10 epoche
    "lr0":          0.001,                    # learning rate iniziale
    "lrf":          0.1,                     # final LR = lr0 * lrf
    "momentum":     0.937,                   # momentum / beta1 per Adam
    "weight_decay": 0.0005,                  # regularizzazione L2
    "warmup_epochs":    3.0,                 # epoche warmup LR
    "warmup_momentum":  0.8,                 # momentum warmup
    "warmup_bias_lr":   0.1,                 # bias LR warmup
    "seed":         42,                      # per riproducibilità
    "fraction":     1.0,                     # usa 100% del dataset
    "val":          True,                    # abilita validazione
    "plots":        True                    # salva grafici di training/val
}

In [4]:
# Parametri di data augmentation ottimizzati
augmentation_params = {
    # Colore (HSV)
    "hsv_h":       0.015,   # Default: 0.015 (range 0.0–1.0) – mantiene piccole variazioni di tonalità
    "hsv_s":       0.7,     # Default: 0.7   (range 0.0–1.0) – variazione consistente di saturazione
    "hsv_v":       0.4,     # Default: 0.4   (range 0.0–1.0) – variazione di luminosità medio-alta

    # Geometriche
    "degrees":     5.0,     # Da 0.0 a 180 – rotazioni lievi per preservare orientamenti realistici
    "translate":   0.1,     # Da 0.0 a 1.0   – spostamenti fino al 10% delle dimensioni d’immagine
    "scale":       0.3,     # ≥ 0.0          – riduzione del range (da 0.5 a 0.3) per minor carico computazionale
    "shear":       0.0,     # –180 a +180    – mantenuto a 0 per evitare distorsioni drastiche
    "perspective": 0.0005,  # 0.0–0.001      – inserita una lieve prospettiva per aumentare la robustezza 3D

    # Flip
    "flipud":      0.0,     # 0.0–1.0 – no flip verticale (rischio di immagini “capovolte” poco realistiche)
    "fliplr":      0.5,     # 0.0–1.0 – flip orizzontale con probabilità 50%

    # Canale BGR
    "bgr":         1.0,     # 0.0–1.0 – scambio completo dei canali per insegnare a gestire l’ordine BGR

    # Mosaic / MixUp / CutMix
    "mosaic":      0.5,     # 0.0–1.0 – mosaic ridotto al 50% per bilanciare velocità e varietà
    "mixup":       0.3,     # 0.0–1.0 – mixup moderato per label noise e greater generalization

    # Classification-only (opzionali, ignora se non serve)
    # "erasing":   0.4,     # 0.0–0.9 – random erasing (classification only, lasciare a default se non serve)
    # "auto_augment": "randaugment",  # classification only
}
train_params.update(augmentation_params)


## Addestramento

In [ ]:
# Addestramento del modello YOLO
# Addestra il modello con gli aumenti dei dati
results = model.train(**train_params)
# Crea la cartella 'results' se non esiste
# Percorso per salvare i file
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

model.save(f"{output_dir}/best_model_{model_name}")  # Salva il modello addestrato

# Nome base dei file
filename = f"{output_dir}/training_results_{model_name.split('.')[0]}"

# Salvataggio in TXT
metrics = results.results_dict if hasattr(results, "results_dict") else {}
with open(f"{filename}.txt", "w") as f:
    f.write(f"Training Results:\n")
    f.write(f"mAP50: {metrics.get('metrics/mAP50(B)', 0.0):.4f}\n")
    f.write(f"mAP50-95: {metrics.get('metrics/mAP50-95(B)', 0.0):.4f}\n")
    f.write(f"Precision: {metrics.get('metrics/precision(B)', 0.0):.4f}\n")
    f.write(f"Recall: {metrics.get('metrics/recall(B)', 0.0):.4f}\n")
    f.write(f"Final Epoch: {getattr(results, 'epoch', 'N/A')}\n")

# Salvataggio in JSON SOLO delle metriche
with open(f"{filename}.json", "w") as f_json:
    json.dump(metrics, f_json, indent=4)

print(f"✅ Risultati di addestramento salvati in '{filename}.txt' e '{filename}.json'")

Ultralytics 8.3.153 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=1.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data/objects365.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.3, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=exp_fastcache, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=2, perspective=0.0005, plots=True, pose=12.0, 

train: Scanning /mnt/e/objects365/labels/train/patch0_a.cache... 93507 images, 0 backgrounds, 0 corrupt: 100%|██████████| 93507/93507 [00:00<?, ?it/s]

train: /mnt/e/objects365/images/train/patch0_a/objects365_v1_00000828.jpg: 1 duplicate labels removed
train: /mnt/e/objects365/images/train/patch0_a/objects365_v1_00002621.jpg: 11 duplicate labels removed
train: /mnt/e/objects365/images/train/patch0_a/objects365_v1_00004592.jpg: 5 duplicate labels removed
train: /mnt/e/objects365/images/train/patch0_a/objects365_v1_00004975.jpg: 1 duplicate labels removed
train: /mnt/e/objects365/images/train/patch0_a/objects365_v1_00006882.jpg: 10 duplicate labels removed
train: /mnt/e/objects365/images/train/patch0_a/objects365_v1_00009196.jpg: 3 duplicate labels removed
train: /mnt/e/objects365/images/train/patch0_a/objects365_v1_00009828.jpg: 3 duplicate labels removed
train: /mnt/e/objects365/images/train/patch0_a/objects365_v1_00012666.jpg: 1 duplicate labels removed
train: /mnt/e/objects365/images/train/patch0_a/objects365_v1_00017147.jpg: 2 duplicate labels removed
train: /mnt/e/objects365/images/train/patch0_a/objects365_v1_00018181.jpg: 7 dup

WARNING ⚠️ train: 436.4GB disk space required, with 50% safety margin but only 225.1/953.9GB free, not caching images to disk
val: Fast image access ✅ (ping: 4.2±1.1 ms, read: 22.7±6.4 MB/s, size: 220.4 KB)


val: Scanning /mnt/e/objects365/labels/val/patch0_a.cache... 25451 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25451/25451 [00:00<?, ?it/s]

...

                   all      25451     493512      0.334      0.225      0.201      0.132

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      16.2G      1.262       1.63       1.22         42        640: 100%|██████████| 2923/2923 [09:09<00:00,  5.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 398/398 [02:50<00:00,  2.33it/s]


                   all      25451     493512      0.336      0.227      0.204      0.135

10 epochs completed in 2.086 hours.
Optimizer stripped from runs/train_yolo11s/exp_fastcache/weights/last.pt, 19.4MB
Optimizer stripped from runs/train_yolo11s/exp_fastcache/weights/best.pt, 19.4MB

Validating runs/train_yolo11s/exp_fastcache/weights/best.pt...
Ultralytics 8.3.153 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO11s summary (fused): 100 layers, 9,554,442 parameters, 0 gradients, 22.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 398/398 [02:52<00:00,  2.31it/s]


                   all      25451     493512      0.336      0.227      0.204      0.135
              Sneakers      12540      34891      0.488      0.668      0.648      0.455
                 Chair       1586       3366       0.25      0.525      0.348      0.219
           Other Shoes       3469       5744      0.273      0.558      0.405      0.293
                   Hat       1562       3264      0.191      0.184     0.0973     0.0448
                   Car       2360       3657      0.305       0.51      0.427      0.291
                  Lamp       1661       3615      0.282      0.414      0.275      0.188
               Glasses       3231       6867      0.305      0.465      0.362      0.244
                Bottle       2368       3019      0.463      0.425      0.411      0.217
                  Desk       4170       9189      0.239      0.417      0.269      0.176
                   Cup       5169       7017      0.321      0.627       0.47      0.332
         Street Light

## Validazione

In [ ]:
import os
from ultralytics import YOLO
import json

# Path al file di pesi salvati
weights_path = f"results/best_model_{model_name}"
data_yaml = "data/objects365.yaml"
# ----------------------------
# 1. Determina se 'model' è già in memoria
# ----------------------------
try:
    # Prova ad accedere a `model`; se non esiste, scatena NameError
    _ = model
    in_ram = True
except NameError:
    in_ram = False

# ----------------------------
# 2. Scegli l'oggetto YOLO per la validazione
# ----------------------------
if in_ram:
    print("✔️ Modello già in RAM: lo uso per la validazione")
    model_to_val = model
elif os.path.exists(weights_path):
    print(f"✔️ Carico modello da disco: {weights_path}")
    model_to_val = YOLO(weights_path)
else:
    raise FileNotFoundError(f"Né modello in RAM né file trovato: {weights_path}")


# Esegui la validazione
val_results = model_to_val.val(
    data=data_yaml,
    imgsz=640,
    batch=4,
    device=0,
    save=True
)

# Crea la cartella dei risultati
os.makedirs("results", exist_ok=True)

# Ottieni il dizionario con le metriche
metrics = val_results.results_dict


filename = f"results/validation_results_{model_name.split('.')[0]}"

# Stampa tutte le chiavi disponibili per debug (facoltativo)
print("Chiavi disponibili in results_dict:")
for k in metrics:
    print(f"- {k}")

# Salvataggio in TXT
with open(f"{filename}.txt", "w") as f:
    f.write("Validation Results:\n")
    f.write(f"mAP50: {metrics.get('metrics/mAP50', 0.0):.4f}\n")
    f.write(f"mAP50-95: {metrics.get('metrics/mAP50-95', 0.0):.4f}\n")
    f.write(f"Precision: {metrics.get('metrics/precision', 0.0):.4f}\n")
    f.write(f"Recall: {metrics.get('metrics/recall', 0.0):.4f}\n")
    f.write(f"Inference Speed: {val_results.speed.get('inference', 0.0):.2f} ms/img\n")

# Salvataggio in JSON
with open(f"{filename}.json", "w") as f_json:
    json.dump(metrics, f_json, indent=4)

print(f"✅ Risultati salvati in '{filename}.txt' e '{filename}.json'")


✔️ Modello già in RAM: lo uso per la validazione
Ultralytics 8.3.153 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO11s summary (fused): 100 layers, 9,554,442 parameters, 0 gradients, 22.1 GFLOPs
val: Fast image access ✅ (ping: 2.6±0.2 ms, read: 19.4±10.3 MB/s, size: 173.5 KB)


val: Scanning /mnt/e/objects365/labels/val/patch0_a.cache... 25451 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25451/25451 [00:00<?, ?it/s]

val: /mnt/e/objects365/images/val/patch0_a/objects365_v1_00006056.jpg: 15 duplicate labels removed
val: /mnt/e/objects365/images/val/patch0_a/objects365_v1_00007269.jpg: 1 duplicate labels removed
val: /mnt/e/objects365/images/val/patch0_a/objects365_v1_00008625.jpg: 3 duplicate labels removed
val: /mnt/e/objects365/images/val/patch0_a/objects365_v1_00016324.jpg: 1 duplicate labels removed
val: /mnt/e/objects365/images/val/patch0_a/objects365_v1_00018594.jpg: 1 duplicate labels removed
val: /mnt/e/objects365/images/val/patch0_b/objects365_v1_00023312.jpg: 25 duplicate labels removed
val: /mnt/e/objects365/images/val/patch0_b/objects365_v1_00025340.jpg: 17 duplicate labels removed
val: /mnt/e/objects365/images/val/patch0_b/objects365_v1_00028356.jpg: 1 duplicate labels removed
val: /mnt/e/objects365/images/val/patch0_b/objects365_v1_00029738.jpg: 1 duplicate labels removed
val: /mnt/e/objects365/images/val/patch0_b/objects365_v1_00031248.jpg: 1 duplicate labels removed
val: /mnt/e/objec


val: Caching images (51.0GB Disk): 100%|██████████| 25451/25451 [00:57<00:00, 445.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6363/6363 [04:34<00:00, 23.19it/s]


                   all      25451     493512      0.316      0.202       0.18      0.117
              Sneakers      12540      34891      0.464      0.635      0.614      0.428
                 Chair       1586       3366      0.228      0.474      0.292      0.181
           Other Shoes       3469       5744      0.257      0.548      0.387      0.279
                   Hat       1562       3264      0.151      0.149     0.0651     0.0295
                   Car       2360       3657      0.252      0.457      0.357      0.238
                  Lamp       1661       3615      0.247      0.364      0.223      0.148
               Glasses       3231       6867      0.304      0.424      0.327      0.217
                Bottle       2368       3019      0.415      0.351      0.335       0.17
                  Desk       4170       9189      0.222      0.355      0.224      0.143
                   Cup       5169       7017      0.314      0.618      0.463      0.335
         Street Light